In [1]:
#execute related notebooks within the scope of the current one to use their variables without the need for individual imports or file creation
#%run dev_pressure_curves.ipynb dev_machine_learning.ipynb

In [2]:
#Here, we import pressure curve files/object to run through a simple workflow. %store -[OPTION] var to -r : retrieve, -d : delete, -z : clear all  
%store -r pc 

In [3]:
methods_plot = pc.plot_methods()
methods_plot.show()

methods = pc.get_methods()
print("Methods: ", methods)

Methods:  ['SAA_411_Doc.M', 'SAA_411_Pac.M', 'SAA_411_5FU.M', 'SAA_411_Irino.M', 'SAA_411_Gem.M']


In [4]:
indices = pc.get_method_indices('SAA_411_Pac.M')
pac_plots = pc.plot_batches(indices)
pac_plots.show()

In [5]:
from src.StreamPort.device.methods import PressureCurvesMethodExtractFeaturesNative

processor = PressureCurvesMethodExtractFeaturesNative(period=10, bins=6, window_size=7)
pc = processor.run(pc)

In [6]:
batches = [batch for batch in pc.get_batches() if "Pac" in batch]
print("Batches: ", batches)

batches.sort()
print("Sorted by date: ", batches)

tests = batches[2:]

Batches:  ['210813_Pac 2021-08-13 10-37-27', '210826_Pac 2021-08-26 16-15-14', '210906_Pac 2021-09-06 11-09-22', '210812_Pac 2021-08-12 10-30-07', '210819_Pac 2021-08-19 15-50-49', '210910_Pac 2021-09-10 14-25-54']
Sorted by date:  ['210812_Pac 2021-08-12 10-30-07', '210813_Pac 2021-08-13 10-37-27', '210819_Pac 2021-08-19 15-50-49', '210826_Pac 2021-08-26 16-15-14', '210906_Pac 2021-09-06 11-09-22', '210910_Pac 2021-09-10 14-25-54']


In [7]:
print("test batch dates: ", tests) # used to get train data, then discarded and test sets iteratively selected by increasing batch date

test batch dates:  ['210819_Pac 2021-08-19 15-50-49', '210826_Pac 2021-08-26 16-15-14', '210906_Pac 2021-09-06 11-09-22', '210910_Pac 2021-09-10 14-25-54']


In [8]:
# create training set
first_test_group = tests[0]
print("First test group date: ", first_test_group)

first_test = first_test_group.split(" ")[-2:]
first_test = " ".join(first_test)
first_test = first_test.replace(":", "-")

date_threshold_min = first_test

First test group date:  210819_Pac 2021-08-19 15-50-49


In [9]:
train_indices = []
for i in indices:
    meta = pc.get_metadata(i)
    batch_position = meta["batch_position"].item()
    start_time = meta["start_time"].item()
    if isinstance(start_time, str):
        start_time = start_time.replace(":", "-")
    else:
        start_time = start_time.strftime("%Y-%m-%d %H-%M-%S")
    if batch_position > 5 and start_time < date_threshold_min:
        train_indices.append(i)

train_data = pc.get_features(train_indices)
train_metadata = pc.get_metadata(train_indices)
train_data.to_csv("dev/workflow_train_features.csv", index=False) # test data files will include batch date
train_metadata.to_csv("dev/workflow_train_metadata.csv", index=False)

train_size = len(train_indices)
print("Number of training curves: ", train_size)

Number of training curves:  18


In [10]:
fig_train=pc.plot_pressure_curves(train_indices)
fig_train.update_layout(showlegend=False)
fig_train.show()
#train_indices.extend([239, 245, 266, 117])# 117 is interesting

In [11]:
fig_train_features = pc.plot_features(train_indices)
fig_train_features.show()

In [12]:
train_data.describe() # maybe this can be fed to model to guide feature selection for splits based on importance. Increase weight of less-impactful statistical features like area

,runtime,area_0.0_0.66,min_0.0_0.66,max_0.0_0.66,mean_0.0_0.66,std_0.0_0.66,range_0.0_0.66,residual_std_0.0_0.66,residual_noise_0.0_0.66,relative_change_0.0_0.66,...,min_3.33_3.975,max_3.33_3.975,mean_3.33_3.975,std_3.33_3.975,range_3.33_3.975,residual_std_3.33_3.975,residual_noise_3.33_3.975,relative_change_3.33_3.975,roc_3.33_3.975,abs_deviation_3.33_3.975
count,18.0,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,...,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000
mean,239.7,45.615564,59.168333,70.964444,69.082222,2.263282,11.796111,0.290950,32.101647,0.191190,...,40.111667,53.816667,46.444145,3.986602,13.705000,0.179988,25.286868,0.336882,0.080908,0.624305
std,0.0,0.138899,0.438718,0.245146,0.210181,0.097076,0.442641,0.028393,2.766722,0.009539,...,0.208616,0.273646,0.093785,0.017303,0.251027,0.021201,2.593377,0.007567,0.065134,0.116780
min,239.7,45.382050,58.140000,70.470000,68.727368,2.127925,11.170000,0.226699,24.606693,0.177064,...,39.760000,53.390000,46.339769,3.956200,13.200000,0.140243,19.756447,0.324318,0.009245,0.419862
25%,239.7,45.535100,58.835000,70.840000,68.959511,2.195601,11.500000,0.275235,31.004150,0.184337,...,39.907500,53.630000,46.380019,3.972435,13.510000,0.166760,23.567972,0.333250,0.036038,0.562356
50%,239.7,45.597825,59.340000,70.940000,69.057180,2.238455,11.680000,0.298989,31.888993,0.190649,...,40.120000,53.745000,46.409308,3.983898,13.750000,0.180211,25.136645,0.334498,0.059937,0.630761
75%,239.7,45.703975,59.470000,71.020000,69.216090,2.329823,12.042500,0.307609,34.192084,0.196815,...,40.250000,53.977500,46.492596,4.003031,13.857500,0.193109,27.761196,0.342449,0.123459,0.661695
max,239.7,45.881100,59.780000,71.510000,69.484060,2.465887,12.720000,0.328231,36.184141,0.213450,...,40.430000,54.360000,46.677385,4.011977,14.110000,0.225321,29.002858,0.350559,0.219686,0.915353


In [13]:
import pandas as pd
from src.StreamPort.machine_learning.analyses import MachineLearningAnalyses

ml_ana = MachineLearningAnalyses(variables = train_data, metadata = train_metadata)
print(ml_ana)

c:\Users\Sandeep\Desktop\StreamPort\sp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html




MachineLearningAnalyses 
  variables: 18 rows, 67 columns
  metadata: 18 rows, 14 columns



In [14]:
from src.StreamPort.machine_learning.methods import MachineLearningScaleFeaturesScalerSklearn

scaler = MachineLearningScaleFeaturesScalerSklearn(scaler_type = "StandardScaler")
ml_ana = scaler.run(ml_ana)

fig_train_features = ml_ana.plot_data()
fig_train_features.update_layout(title="Train set features")
fig_train_features.show()

In [15]:

from src.StreamPort.machine_learning.methods import MachineLearningMethodIsolationForestSklearn

iforest = MachineLearningMethodIsolationForestSklearn()

ml_ana = iforest.run(ml_ana)


ml_ana.train()
fig_train_scores = ml_ana.plot_scores()
fig_train_scores.show()

In [16]:
# new tests sorted by date
tests = {}
for batch in batches:
    batch_date = batch.split(" ")[-2 : ]
    batch_date = " ".join(batch_date)

    if batch_date >= date_threshold_min: # search and collect test samples
        date_threshold_min = batch_date 
        
        test_indices = pc.get_batch_indices(batch)
        test_size = len(test_indices)

        for i in range(test_size):
            tests[f"{batch_date}_{i+1}"] = test_indices[i]
            
    else:
        continue

In [17]:
dates = list(tests.keys())
print(dates)

['2021-08-19 15-50-49_1', '2021-08-19 15-50-49_2', '2021-08-19 15-50-49_3', '2021-08-19 15-50-49_4', '2021-08-19 15-50-49_5', '2021-08-19 15-50-49_6', '2021-08-19 15-50-49_7', '2021-08-19 15-50-49_8', '2021-08-19 15-50-49_9', '2021-08-19 15-50-49_10', '2021-08-19 15-50-49_11', '2021-08-19 15-50-49_12', '2021-08-19 15-50-49_13', '2021-08-19 15-50-49_14', '2021-08-19 15-50-49_15', '2021-08-19 15-50-49_16', '2021-08-19 15-50-49_17', '2021-08-26 16-15-14_1', '2021-08-26 16-15-14_2', '2021-08-26 16-15-14_3', '2021-08-26 16-15-14_4', '2021-08-26 16-15-14_5', '2021-08-26 16-15-14_6', '2021-08-26 16-15-14_7', '2021-08-26 16-15-14_8', '2021-08-26 16-15-14_9', '2021-08-26 16-15-14_10', '2021-08-26 16-15-14_11', '2021-08-26 16-15-14_12', '2021-08-26 16-15-14_13', '2021-08-26 16-15-14_14', '2021-08-26 16-15-14_15', '2021-08-26 16-15-14_16', '2021-08-26 16-15-14_17', '2021-08-26 16-15-14_18', '2021-08-26 16-15-14_19', '2021-08-26 16-15-14_20', '2021-09-06 11-09-22_1', '2021-09-06 11-09-22_2', '2021

In [91]:
#for date in dates[:1]:
date = dates[13]

test_data = pc.get_features(tests[date])
test_metadata = pc.get_metadata(tests[date])

print(f"Test {date}: index:", tests[date])

fig_test_features = pc.plot_features(tests[date], normalize=False)
fig_test_features.show()

fig_test_features_raw = pc.plot_features_raw(tests[date])
fig_test_features_raw.update_layout(showlegend=False)
fig_test_features_raw.show()

Test 2021-08-19 15-50-49_14: index: 129


In [86]:
ml_ana.predict(test_data, test_metadata)

outliers = ml_ana.test_prediction_outliers() # defaults: n_tests = 1, show_scores = False
print(outliers)

fig_test_curves = pc.plot_pressure_curves(test_metadata["index"].tolist())
fig_test_curves.show()

fig_test_scores = ml_ana.plot_scores()
fig_test_scores.update_layout(title=f"Test set {date} final run")
fig_test_scores.show()


Setting true_classes to majority class...
   index  batch_position  train_size  train_time  threshold     score  \
0    128              13          27    0.383789   -0.04903 -0.030318   

   confidence    class  
0        0.62  outlier  


In [87]:
#confidence_plot = evaluator.plot_confidences()
confidence_plot = ml_ana.plot_confidences()
confidence_plot.show()

In [88]:
import os

test_data.drop(columns="runtime", inplace=True)
data = pd.concat([test_metadata, test_data], axis=1)

if outliers["class"].iloc[0] == "outlier":
    outlier_data = data

    if os.path.exists("dev/outliers.csv"):
        outliers_file = pd.read_csv("dev/outliers.csv")
        outliers_file = pd.concat([outliers_file, outlier_data], axis = 0)
        outliers_file.drop_duplicates(subset="index", inplace = True, ignore_index=True)
        outliers_file.to_csv("dev/outliers.csv", index=False)
    
    else:
        outlier_data.to_csv("dev/outliers.csv", index=False)

else:

    if os.path.exists("dev/normals.csv"):
        normals_file = pd.read_csv("dev/normals.csv")

        normals_file = pd.concat([normals_file, data], axis = 0)
        normals_file.drop_duplicates(subset="index", inplace=True, ignore_index=True)
        normals_file.to_csv("dev/normals.csv", index=False)
    
    else:
        data.to_csv("dev/normals.csv", index=False)

ml_ana.add_prediction() # add_data already calls self.train()

In [89]:
#threshold_plot = evaluator.plot_threshold_variation()
threshold_plot = ml_ana.plot_threshold_variation()
threshold_plot.show()
#threshold_plot.write_image("dev/figures/fig_threshold_variation_serialized_tests.png", width=1100, height= 350, scale = 3)

In [90]:
#train_time_plot = evaluator.plot_train_time()
train_time_plot = ml_ana.plot_train_time()
train_time_plot.show()